In [1]:
%pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
gpu_available = torch.cuda.is_available()
print(f"GPU Available: {gpu_available}")

# If GPU is available, print the GPU name
if gpu_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

# Check if tensors are being moved to GPU
device = torch.device("cuda" if gpu_available else "cpu")
tensor = torch.rand(3, 3).to(device)
print(f"Tensor is on device: {tensor.device}")


GPU Available: False
Tensor is on device: cpu


In [3]:
# Create the CartPole-v1 environment
env = gym.make("CartPole-v1")

# Reset the environment to obtain the initial state
state, info = env.reset()

In [4]:
"""
Part 1.1: State Discretization
"""

# Define the number of bins for each state component
num_bins = 10

# Define bin ranges for each state component based on typical CartPole state ranges
cart_position_range = [-2.4, 2.4]
cart_velocity_range = [-2.0, 2.0]
pole_angle_range = [-0.2095, 0.2095]  # Roughly ±12 degrees in radians
pole_velocity_range = [-3.0, 3.0]

# Custom mapping function to discretize continuous state values
def custom_discretize(value, value_range, num_bins):
    """
    Discretize a continuous value using the provided value range and number of bins.
    """
    # Extract the min and max values from the value range
    min_val, max_val = value_range

    # Check if value is below min range or above max range, and clamp it
    if value <= min_val:
        return 0
    elif value >= max_val:
        return num_bins - 1

    # Calculate the relative position of value within its range
    bin_width = (max_val - min_val) / num_bins
    bin_index = int((value - min_val) / bin_width)
    return bin_index

In [5]:
def discretize_state(state):
    """
    Discretize a state into a tuple of bin indices based on the bin ranges and number of bins defined above.
    """
    # Unpack the state components
    cart_pos, cart_vel, pole_angle, pole_vel = state

    # Map each component of the state to a discrete bin index using the custom function
    cart_pos_discrete = custom_discretize(cart_pos, cart_position_range, num_bins)
    cart_vel_discrete = custom_discretize(cart_vel, cart_velocity_range, num_bins)
    pole_angle_discrete = custom_discretize(pole_angle, pole_angle_range, num_bins)
    pole_vel_discrete = custom_discretize(pole_vel, pole_velocity_range, num_bins)

    # Return the discrete state as a tuple of bin indices
    return (cart_pos_discrete, cart_vel_discrete, pole_angle_discrete, pole_vel_discrete)

In [6]:
"""
Discretize the initial state
"""

# Gather states and their discretized versions
continuous_states = []
discretized_states = []

# Run the environment and collect state samples
for episode in range(20):  # Collect data from 20 episodes
    state, _ = env.reset()  # Reset environment and get initial state
    done = False
    
    while not done:
        # Save continuous state for plotting
        continuous_states.append(state)
        
        # Discretize the state and save it
        discretized_states.append(discretize_state(state))
        
        # Take a random action and step through the environment
        action = env.action_space.sample()
        state, _, terminated, truncated, _ = env.step(action)
        
        # Check if the episode is over
        done = terminated or truncated

# Convert the collected states to NumPy arrays for easy manipulation
continuous_states = np.array(continuous_states)
discretized_states = np.array(discretized_states)

# Print the continuous and discretized states
print('continuous_states', continuous_states)
print('discretized_states', discretized_states)


continuous_states [[ 0.02642061  0.02536744  0.00326753 -0.02269654]
 [ 0.02692796 -0.16980122  0.0028136   0.27101552]
 [ 0.02353194  0.02528047  0.00823391 -0.02077864]
 ...
 [-0.03472318 -0.833149    0.15159914  1.3649452 ]
 [-0.05138616 -0.6402155   0.17889804  1.1232638 ]
 [-0.06419047 -0.8371734   0.20136331  1.4663014 ]]
discretized_states [[5 5 5 4]
 [5 4 5 5]
 [5 5 5 4]
 ...
 [4 2 8 7]
 [4 3 9 6]
 [4 2 9 7]]


In [7]:
"""
Part 1.3: Implementing Temporal Difference Learning - SARSA & Q-Learning
"""

# Define the action space and Q-table dimensions
num_actions = env.action_space.n  # Number of actions in CartPole (left, right)
state_space_size = (num_bins,) * 4  # 4-tuple of number of bins for each state component

In [8]:
def choose_action(action, state, exploration_rate=0.5):
    """
    Epsilon-greedy policy with a small chance of exploration.
    """
    if np.random.rand() < exploration_rate:
        # Explore: Choose a random action
        return np.random.randint(num_actions)
    else:
        # Exploit: Choose the action with the highest Q-value
        return np.argmax(action[state])

In [9]:
def td_control(env, num_episodes, Q_table, exploration_rate, learning_rate, discount_factor, method):
    """
    Temporal Difference Learning algorithm for Q-Learning and SARSA.
    """
    for episode in range(num_episodes):
        # Reset the environment and discretize the initial state
        state, _ = env.reset()
        discrete_state = discretize_state(state)

        total_reward = 0  # Track total reward in each episode
        total_td_error = 0  # Track total TD error in each episode

        done = False
        while not done:
            # Choose the action using the epsilon-greedy policy
            action = choose_action(Q_table, discrete_state, exploration_rate)

            # Take the chosen action and observe the next state and reward
            next_state, reward, terminated, truncated, _ = env.step(action)
            next_discrete_state = discretize_state(next_state)

            if method == "sarsa":
                # SARSA: Choose the next action using the current policy (on-policy)
                next_action = choose_action(Q_table, next_discrete_state, exploration_rate)
                td_target = reward + discount_factor * Q_table[next_discrete_state][next_action]

            elif method == "q_learning":
                # Q-Learning: Use the greedy action for the next state (off-policy)
                best_next_action = np.argmax(Q_table[next_discrete_state])
                td_target = reward + discount_factor * Q_table[next_discrete_state][best_next_action]

            else:
                raise ValueError("Method must be 'sarsa' or 'q_learning'")

            # Calculate the TD error and update the Q-value
            td_error = td_target - Q_table[discrete_state][action]
            Q_table[discrete_state][action] += learning_rate * td_error

            # Accumulate total reward and TD error
            total_reward += reward
            total_td_error += abs(td_error)

            # Move to the next state
            discrete_state = next_discrete_state

            # Check if episode is done
            done = terminated or truncated

        # Print metrics every 1000 episodes
        if (episode + 1) % 1000 == 0:
            print(f"Episode {episode + 1}: \tTotal Reward = {total_reward:.2f}, "
                  f"\tTotal TD Error = {total_td_error:.4f}")

In [10]:
"""
Implementing Temporal Difference Learning - Expected SARSA
"""
# Define necessary parameters
num_episodes = 20000
exploration_rate = 0.1  # Epsilon for epsilon-greedy strategy
learning_rate = 0.01  # Alpha, step size for Q-value updates
discount_factor = 0.99  # Gamma, discount factor for future rewards

# Initialize the Q-table for Expected SARSA
Q_sarsa = np.zeros((num_bins, num_bins, num_bins, num_bins, num_actions))

# Call the td_control function for SARSA
td_control(env, num_episodes, Q_sarsa, exploration_rate, learning_rate, discount_factor, method="sarsa")


Episode 1000: 	Total Reward = 11.00, 	Total TD Error = 16.8553
Episode 2000: 	Total Reward = 9.00, 	Total TD Error = 16.6455
Episode 3000: 	Total Reward = 9.00, 	Total TD Error = 14.6691
Episode 4000: 	Total Reward = 9.00, 	Total TD Error = 20.3293
Episode 5000: 	Total Reward = 16.00, 	Total TD Error = 30.9929
Episode 6000: 	Total Reward = 9.00, 	Total TD Error = 9.8529
Episode 7000: 	Total Reward = 9.00, 	Total TD Error = 12.0819
Episode 8000: 	Total Reward = 9.00, 	Total TD Error = 21.2218
Episode 9000: 	Total Reward = 11.00, 	Total TD Error = 15.0077
Episode 10000: 	Total Reward = 10.00, 	Total TD Error = 13.3733
Episode 11000: 	Total Reward = 9.00, 	Total TD Error = 15.5492
Episode 12000: 	Total Reward = 10.00, 	Total TD Error = 37.3264
Episode 13000: 	Total Reward = 10.00, 	Total TD Error = 20.4622
Episode 14000: 	Total Reward = 10.00, 	Total TD Error = 11.7133
Episode 15000: 	Total Reward = 8.00, 	Total TD Error = 20.9342
Episode 16000: 	Total Reward = 10.00, 	Total TD Error = 11

In [11]:
"""
Implementing Temporal Difference Learning - Q-Learning
"""
# Define necessary parameters
num_episodes = 20000
exploration_rate = 0.1  # Epsilon for epsilon-greedy strategy
learning_rate = 0.01  # Alpha, step size for Q-value updates
discount_factor = 0.99  # Gamma, discount factor for future rewards

# Initialize the Q-table for Q-Learning
Q_qlearning = np.zeros((num_bins, num_bins, num_bins, num_bins, num_actions))

# Call the td_control function for Q-Learning
td_control(env, num_episodes, Q_qlearning, exploration_rate, learning_rate, discount_factor, method="q_learning")


Episode 1000: 	Total Reward = 8.00, 	Total TD Error = 11.3145
Episode 2000: 	Total Reward = 10.00, 	Total TD Error = 19.6079
Episode 3000: 	Total Reward = 9.00, 	Total TD Error = 22.1948
Episode 4000: 	Total Reward = 10.00, 	Total TD Error = 13.0170
Episode 5000: 	Total Reward = 10.00, 	Total TD Error = 13.6860
Episode 6000: 	Total Reward = 9.00, 	Total TD Error = 13.0838
Episode 7000: 	Total Reward = 10.00, 	Total TD Error = 15.4054
Episode 8000: 	Total Reward = 10.00, 	Total TD Error = 13.4970
Episode 9000: 	Total Reward = 10.00, 	Total TD Error = 20.9820
Episode 10000: 	Total Reward = 10.00, 	Total TD Error = 16.3825
Episode 11000: 	Total Reward = 8.00, 	Total TD Error = 24.1020
Episode 12000: 	Total Reward = 10.00, 	Total TD Error = 17.4058
Episode 13000: 	Total Reward = 17.00, 	Total TD Error = 86.6202
Episode 14000: 	Total Reward = 9.00, 	Total TD Error = 12.4197
Episode 15000: 	Total Reward = 10.00, 	Total TD Error = 17.6879
Episode 16000: 	Total Reward = 9.00, 	Total TD Error =

In [12]:
"""
Part 1.4: Implementing Monte Carlo Prediction
"""

# Initialize the value function and state counts to zeros using numpy arrays
V = np.zeros((num_bins, num_bins, num_bins, num_bins))
state_count = np.zeros((num_bins, num_bins, num_bins, num_bins))

# Hyperparameters
discount_factor = 0.99      # Discount factor for calculating returns
num_episodes = 10000        # Number of episodes to run for MC prediction

In [13]:
def monte_carlo_prediction(env, num_episodes, discount_factor):
    """
    Perform first-visit Monte Carlo prediction to estimate the state value function
    """
    # Loop over the episodes
    for episode_num in range(num_episodes):
        # Generate an episode using the random policy
        episode = []  # List to store (state, reward) tuples
        state, _ = env.reset()
        done = False
        
        # Track states that have been visited in the episode for first-visit MC
        while not done:
            # Random action policy
            discrete_state = discretize_state(state)
            action = choose_action(V, discrete_state, exploration_rate)
            next_state, reward, terminated, truncated, _ = env.step(action)
            
            # Discretize the state and store it along with the reward in the episode
            episode.append((discrete_state, reward))
            
            # Move to the next state
            state = next_state
            done = terminated or truncated
        
        # Track states that have been visited in the episode for first-visit MC
        visited_states = set()
        
        # Calculate returns for each state in the episode
        G = 0  # Return value, cumulative discounted reward
        for t in reversed(range(len(episode))):
            state, reward = episode[t]
            G = discount_factor * G + reward  # Calculate return G_t

            # First-visit MC: Check if this is the first visit to the state in the episode
            if state not in visited_states:
                visited_states.add(state)
                
                # Update the state value estimate using incremental formula
                state_count[state] += 1
                V[state] += (G - V[state]) / state_count[state]  # Incremental mean update formula

        # Print progress every 50 episodes
        if (episode_num + 1) % 1000 == 0:
            print(f"Episode {episode_num + 1}/{num_episodes} completed.")

In [14]:
# Run Monte Carlo Prediction
monte_carlo_prediction(env, num_episodes, discount_factor)

# Print final state value estimates for some sample states
sample_states = [(1,1,1,1),(2,2,2,2),(3,3,3,3),(4,4,4,4),(5,5,5,5),(6,6,6,6)]
for state in sample_states:
    print(f"State {state} - Estimated Value/Return: {V[state]:.2f}")

# Close the environment after training
env.close()

Episode 1000/10000 completed.
Episode 2000/10000 completed.
Episode 3000/10000 completed.
Episode 4000/10000 completed.
Episode 5000/10000 completed.
Episode 6000/10000 completed.
Episode 7000/10000 completed.
Episode 8000/10000 completed.
Episode 9000/10000 completed.
Episode 10000/10000 completed.
State (1, 1, 1, 1) - Estimated Value/Return: 0.00
State (2, 2, 2, 2) - Estimated Value/Return: 0.00
State (3, 3, 3, 3) - Estimated Value/Return: 0.00
State (4, 4, 4, 4) - Estimated Value/Return: 9.89
State (5, 5, 5, 5) - Estimated Value/Return: 8.74
State (6, 6, 6, 6) - Estimated Value/Return: 0.00
